# import libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import numpy as np
from collections import Counter
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
drive_path = '/content/drive/MyDrive/UTS_MKTK'

Mounted at /content/drive/


# load dataset: CIFAR-10

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

170508288/170498071 [==============================] - 2s 0us/step


filter only airplane, automobile, ship, and truck. each category has 800 data

In [ ]:
for i in [0, 1, 8, 9]:
  idx = (train_labels == i).reshape(train_images.shape[0])
  train_img = train_images[idx][:500]
  val_img = train_images[idx][500:750]
  test_img = train_images[idx][750:800]
  if i == 0:
    airplane_img_tr = train_img
    airplane_img_val = val_img
    airplane_img_te = test_img    
  elif i == 1:
    automobile_img_tr = train_img
    automobile_img_val = val_img
    automobile_img_te = test_img
  elif i == 8:
    ship_img_tr = train_img
    ship_img_val = val_img
    ship_img_te = test_img
  else:
    truck_img_tr = train_img
    truck_img_val = val_img
    truck_img_te = test_img

training_image = np.concatenate((airplane_img_tr, automobile_img_tr, ship_img_tr, truck_img_tr), axis=0)
validation_image = np.concatenate((airplane_img_val, automobile_img_val, ship_img_val, truck_img_val), axis=0)
testing_image = np.concatenate((airplane_img_te, automobile_img_te, ship_img_te, truck_img_te), axis=0)

In [ ]:
A = []
B = []
C = []
D = []
for i in range(500):
  A.append([0])
  B.append([1])
  C.append([2])
  D.append([3])

A = np.array(A)
B = np.array(B)
C = np.array(C)
D = np.array(D)

training_label = np.concatenate((A, B, C, D), axis=0)

In [ ]:
A = []
B = []
C = []
D = []
for i in range(250):
  A.append([0])
  B.append([1])
  C.append([2])
  D.append([3])

A = np.array(A)
B = np.array(B)
C = np.array(C)
D = np.array(D)

validation_label = np.concatenate((A, B, C, D), axis=0)

In [ ]:
A = []
B = []
C = []
D = []
for i in range(50):
  A.append([0])
  B.append([1])
  C.append([2])
  D.append([3])

A = np.array(A)
B = np.array(B)
C = np.array(C)
D = np.array(D)

testing_label = np.concatenate((A, B, C, D), axis=0)

In [ ]:
print(training_image.shape)
print(validation_image.shape)
print(testing_image.shape)

print(training_label.shape)
print(validation_label.shape)
print(testing_label.shape)

class_names = ['airplane', 'automobile', 'ship', 'truck']

(2000, 32, 32, 3)
(1000, 32, 32, 3)
(200, 32, 32, 3)
(2000, 1)
(1000, 1)
(200, 1)


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((training_image, training_label))
test_ds = tf.data.Dataset.from_tensor_slices((testing_image, testing_label))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_image, validation_label))

# define Preprocessing

In [ ]:
def process_images(image, label):
    image = tf.image.per_image_standardization(image) #strandarisasi
    image = tf.image.resize(image, (224,224))         #resize
    return image, label

In [ ]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 2000
Test data size: 200
Validation data size: 1000


In [ ]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=10, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=10, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=10, drop_remainder=True))

# model

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4, activation='softmax')
])

In [ ]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()

#Defining callbacks to get the best model possible
my_calls = [keras.callbacks.EarlyStopping(monitor='val_loss',patience=50),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=10, min_lr=1.5e-5),
            keras.callbacks.ModelCheckpoint(drive_path + "/Model1.h5",verbose=1,save_best_only=True),
            keras.callbacks.TensorBoard(run_logdir)]

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.Adam(lr=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 54, 54, 96)        34944     
                                                                 
 batch_normalization (BatchN  (None, 54, 54, 96)       384       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 26, 26, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 batch_normalization_1 (Batc  (None, 26, 26, 256)      1024      
 hNormalization)                                                 
                                                        

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_ds,
          epochs=100,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=my_calls)

Epoch 1/100
200/200 [==============================] - ETA: 0s - loss: 9.0778 - accuracy: 0.3820
Epoch 00001: val_loss improved from inf to 1.57699, saving model to /content/drive/MyDrive/UTS_MKTK/Model1.h5
200/200 [==============================] - 49s 85ms/step - loss: 9.0778 - accuracy: 0.3820 - val_loss: 1.5770 - val_accuracy: 0.4310 - lr: 0.0010
Epoch 2/100
200/200 [==============================] - ETA: 0s - loss: 1.6915 - accuracy: 0.4185
Epoch 00002: val_loss improved from 1.57699 to 1.18458, saving model to /content/drive/MyDrive/UTS_MKTK/Model1.h5
200/200 [==============================] - 18s 87ms/step - loss: 1.6915 - accuracy: 0.4185 - val_loss: 1.1846 - val_accuracy: 0.4580 - lr: 0.0010
Epoch 3/100
200/200 [==============================] - ETA: 0s - loss: 1.5269 - accuracy: 0.4310
Epoch 00003: val_loss improved from 1.18458 to 1.16258, saving model to /content/drive/MyDrive/UTS_MKTK/Model1.h5
200/200 [==============================] - 21s 100ms/step - loss: 1.5269 - accu

# load model and evaluate

In [ ]:
model = load_model(drive_path + "/Model1.h5")

In [ ]:
model.evaluate(test_ds)

20/20 [==============================] - 29s 22ms/step - loss: 0.7335 - accuracy: 0.7150


[0.7334667444229126, 0.7149999737739563]